In [3]:
library(tidyverse)
library(lubridate)
library(RPostgreSQL)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Loading required package: DBI


In [ ]:
file_dir <- "/Data/common"

In [ ]:
mort <- data.table::fread(file.path(file_dir, "mort_sum_full.csv"), data.table = FALSE)
head(mort)

In [ ]:
vent <- data.table::fread(file.path(file_dir, "Ventilator_vent.tsv"))
head(vent)

In [ ]:
drv <- dbDriver("PostgreSQL")
user <- pw <- "datathon"
dbname <- "mimic"
con <- dbConnect(drv, dbname = dbname, user = user, password = pw)

In [ ]:
meta <- dbGetQuery(con, "SELECT icustay_id FROM icustays WHERE dbsource = 'metavision'")
head(meta)

In [ ]:
fst_icu <- data.table::fread("/Data/common/first_icu_id.csv", data.table = FALSE)
head(fst_icu)

In [ ]:
mort_meta <- mort %>% inner_join(meta)
head(mort_meta)

In [ ]:
mort_meta_fst <- mort_meta %>% inner_join(fst_icu)

In [ ]:
mort_vent <- vent %>% left_join(mort)
head(mort_vent)

In [ ]:
mort_vent_grp <- mort_vent %>%
    mutate(tv_pbw_grp = case_when(
        is.na(tv_pbw) ~ NA_integer_,
        tv_pbw < 6 ~ 1L,
        (tv_pbw < 8 & tv_pbw >= 6) ~ 2L,
        (tv_pbw < 10 & tv_pbw >= 8) ~ 3L,
        tv_pbw >= 10 ~ 4L
    ))
head(mort_vent_grp)

In [ ]:
mort_vent_grp %>% pull(tv_pbw_grp) %>% table(useNA = "ifany")

In [ ]:
mort_vent_fst <- mort_vent %>% inner_join(fst_icu)
head(mort_vent_fst)

In [ ]:
mort_vent_fst_grp <- mort_vent_fst %>%
    mutate(tv_pbw_grp = case_when(
        is.na(tv_pbw) ~ NA_integer_,
        tv_pbw < 6 ~ 1L,
        (tv_pbw < 8 & tv_pbw >= 6) ~ 2L,
        (tv_pbw < 10 & tv_pbw >= 8) ~ 3L,
        tv_pbw >= 10 ~ 4L
    ))
head(mort_vent_fst_grp)

In [ ]:
mort_vent_fst_grp %>% inner_join(meta) %>% pull(tv_pbw_grp) %>% table(useNA = "ifany")

In [ ]:
meta_vent <- meta %>% inner_join(vent) %>%
    mutate(tv_pbw_grp = case_when(
        is.na(tv_pbw) ~ NA_integer_,
        tv_pbw < 6 ~ 1L,
        (tv_pbw < 8) ~ 2L,
        (tv_pbw < 10) ~ 3L,
        TRUE ~ 4L
    ))
meta_vent %>%
    pull(tv_pbw_grp) %>% table(useNA = "ifany")

In [ ]:
nrow(mort)
nrow(meta)

In [ ]:
setdiff(mort_vent$subject_id, meta_vent$subject_id) %>% length

In [ ]:
dbDisconnect(con)
dbUnloadDriver(drv)

In [1]:
x = data.frame(a = 1:3, b = 2:4, c = 3:5)

In [2]:
x

a,b,c
1,2,3
2,3,4
3,4,5


In [10]:
x %>% select_(.dots = c('b','c') %>% sprintf('-%s', .) %>% as.list)

a
1
2
3


In [8]:
letters

[1] "a" "b" "c" "d" "e" "f" "g" "h" "i" "j" "k" "l" "m" "n" "o" "p" "q" "r" "s"
[20] "t" "u" "v" "w" "x" "y" "z"

In [ ]:
c